## Visibility analysis

The visibility analysis will, at first, be conducted using the "Arealstatistik" (https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik/boden-nutzung-bedeckung-eignung/arealstatistik-schweiz.assetdetail.25885691.html).
At a later stage it might be conducted using a DEM.

In [3]:
# setup
import geopandas as gpd
import pandas as pd
import numpy as np


In [6]:
# load data
